# pseudotime-informed NMF - <span style="color:red">Polioudakis 2019 - 4k</span>

**CellOracle environment**

In [ ]:
import scanpy as sc

import celloracle as co

import pandas as pd

import seaborn as sns

import numpy as np

import matplotlib.pyplot as plt

# visualization settings
%config InlineBackend.figure_format = 'retina'
%matplotlib inline

plt.rcParams['figure.figsize'] = (15,7)
plt.rcParams["savefig.dpi"] = 600

# Polioudakis et al 2019 dataset

## Loading raw data

In [ ]:
adata = sc.read_h5ad("/home/jovyan/jm_jlab/data_indNeuro/intermediate_files/pol19_counts.h5ad")

milestones = pd.read_csv("/home/jovyan/jm_jlab/data_indNeuro/scFates_output/polioudakis19_scFates_clustering.tsv", sep='\t', index_col=0)

adata.obs = pd.concat([adata.obs, milestones], axis=1)


#Including embeddings
cellembeddings = pd.read_csv("/home/jovyan/jm_jlab/data_indNeuro/intermediate_files/integrated_polioudakis19_cellembeddings.tsv", sep='\t')

cellembeddings.iloc[:,0] = cellembeddings.iloc[:,0]*-1 #for consistent better geometry

cellembeddings.iloc[:,1] = cellembeddings.iloc[:,1]*-1 #for consistent better geometry

cellembeddings = cellembeddings[cellembeddings.index.str.contains("pol19")]

cellembeddings.index = pd.DataFrame(cellembeddings.index)[0].str.split("pol19_", expand=True)[1]

adata = adata[adata.obs.index.isin(cellembeddings.index),:].copy()

cellembeddings = cellembeddings[cellembeddings.index.isin(adata.obs.index)].copy()

cellembeddings = cellembeddings.reindex(adata.obs.index)

adata.obsm['X_pca'] = cellembeddings.to_numpy() #sharing embeddings

In [ ]:
adata.obs['PC_1'] = adata.obsm['X_pca'][:,0]
adata.obs['PC_2'] = adata.obsm['X_pca'][:,1]

## oRG

In [ ]:
branch_oRG = adata[(adata.obs['segmentation'] == 's_vRG') | (adata.obs['segmentation'] == 's_oRG'),:]

sc.pp.filter_genes(branch_oRG, min_cells=25)

sc.pp.highly_variable_genes(branch_oRG, n_top_genes=4000, flavor='seurat_v3')

branch_oRG = branch_oRG[:,branch_oRG.var['highly_variable'] == True].copy()

my_data = pd.DataFrame(branch_oRG.X.todense(), index=branch_oRG.obs['t'], columns=branch_oRG.var_names)

my_data.sort_index(inplace=True, ascending=True)

for_NMF = sc.AnnData(my_data)

for_NMF

#Including metadata to final object
branch_oRG.obs.index = branch_oRG.obs['t'].astype(np.float64)

branch_oRG.obs = branch_oRG.obs.reindex(np.sort(branch_oRG.obs.index))

for_NMF.obs.index = for_NMF.obs.index.astype(np.float64)

for_NMF.obs = for_NMF.obs.join(branch_oRG.obs)

for_NMF

In [ ]:
for_NMF.write("/home/jovyan/jm_jlab/data_indNeuro/2.NMF/polioudakis19_vRGtooRG_4k.h5ad")

## IPC

In [ ]:
branch_IPC = adata[(adata.obs['segmentation'] == 's_vRG') | (adata.obs['segmentation'] == 's_IPC'),:]

sc.pp.filter_genes(branch_IPC, min_cells=25)

sc.pp.highly_variable_genes(branch_IPC, n_top_genes=4000, flavor='seurat_v3') 

branch_IPC = branch_IPC[:,branch_IPC.var['highly_variable'] == True].copy()

my_data = pd.DataFrame(branch_IPC.X.todense(), index=branch_IPC.obs['t'], columns=branch_IPC.var_names)

my_data.sort_index(inplace=True, ascending=True)

In [ ]:
for_NMF = sc.AnnData(my_data)

for_NMF

In [ ]:
#Including metadata to final object
branch_IPC.obs.index = branch_IPC.obs['t'].astype(np.float64)

branch_IPC.obs = branch_IPC.obs.reindex(np.sort(branch_IPC.obs.index))

for_NMF.obs.index = for_NMF.obs.index.astype(np.float64)

for_NMF.obs = for_NMF.obs.join(branch_IPC.obs)

In [ ]:
for_NMF.write("/home/jovyan/jm_jlab/data_indNeuro/2.NMF/polioudakis19_vRGtoIPC_4k.h5ad")